# Importações, decorators e começo

In [ ]:
import time
import random
import itertools
import math
from functools import wraps, lru_cache
from collections import defaultdict, deque
from typing import Dict, List, Tuple, Set

# Dataset

In [ ]:
SKILLS = {
    "S1": {"Nome": "Programação Básica (Python)", "T": 80,  "V": 3,  "C": 4, "Pre_Reqs": []},
    "S2": {"Nome": "Modelagem de Dados (SQL)", "T": 60,  "V": 4,  "C": 3, "Pre_Reqs": []},
    "S3": {"Nome": "Algoritmos Avançados", "T": 100, "V": 7,  "C": 8, "Pre_Reqs": ["S1"]},
    "S4": {"Nome": "Fundamentos de ML", "T": 120, "V": 8,  "C": 9, "Pre_Reqs": ["S1","S3"]},
    "S5": {"Nome": "Visualização de Dados (BI)", "T": 40,  "V": 6,  "C": 5, "Pre_Reqs": ["S2"]},
    "S6": {"Nome": "IA Generativa Ética", "T": 150, "V": 10, "C": 10,"Pre_Reqs": ["S4"]},
    "S7": {"Nome": "Estruturas em Nuvem (AWS/Azure)", "T": 70, "V": 5, "C": 7, "Pre_Reqs": []},
    "S8": {"Nome": "APIs e Microsserviços", "T": 90,  "V": 6,  "C": 6, "Pre_Reqs": ["S1"]},
    "S9": {"Nome": "DevOps & CI/CD", "T": 110, "V": 9,  "C": 8, "Pre_Reqs": ["S7","S8"]},
    "H10": {"Nome": "Segurança de Dados", "T": 60,  "V": 5,  "C": 6, "Pre_Reqs": []},
    "H11": {"Nome": "Análise de Big Data", "T": 90,  "V": 8,  "C": 8, "Pre_Reqs": ["S4"]},
    "H12": {"Nome": "Introdução a IoT", "T": 30,  "V": 3,  "C": 3, "Pre_Reqs": []}
}

CRITICAL = ["S3","S5","S7","S8","S9"]
TARGET = "S6"

In [ ]:
def timer(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        t0 = time.time()
        res = func(*args, **kwargs)
        t1 = time.time()
        print(f"[TIMER] {func.__name__} levou {t1-t0:.4f}s")
        return res
    return wrapper

def memoize(func):
    cache = {}
    @wraps(func)
    def wrapper(*args):
        key = args
        if key in cache:
            return cache[key]
        val = func(*args)
        cache[key] = val
        return val
    wrapper._cache = cache
    return wrapper

# Grafo

In [ ]:
class GraphValidator:
    def __init__(self, skills_dict):
        self.skills = skills_dict
        self.adj = defaultdict(list)
        for node, meta in self.skills.items():
            for p in meta["Pre_Reqs"]:
                self.adj[p].append(node)
        for node in self.skills.keys():
            self.adj[node]

    def find_orphans(self) -> List[str]:
        orphans = []
        for node, meta in self.skills.items():
            for p in meta["Pre_Reqs"]:
                if p not in self.skills:
                    orphans.append((node, p))
        return orphans

    def detect_cycle(self) -> Tuple[bool, List[str]]:
        visited = set()
        rec_stack = set()
        cycle_nodes = []

        def dfs(u):
            visited.add(u)
            rec_stack.add(u)
            for v in self.skills.get(u, {}).get("Pre_Reqs", []):
                if v not in self.skills:
                    continue
                if v not in visited:
                    if dfs(v):
                        cycle_nodes.append(v)
                        return True
                elif v in rec_stack:
                    cycle_nodes.append(v)
                    return True
            rec_stack.remove(u)
            return False

        for n in self.skills:
            if n not in visited:
                if dfs(n):
                    return True, cycle_nodes[::-1]
        return False, []

    def topological_sort(self) -> List[str]:
        indeg = {n: 0 for n in self.skills}
        graph = defaultdict(list)
        for node, meta in self.skills.items():
            for p in meta["Pre_Reqs"]:
                if p in self.skills:
                    graph[p].append(node)
                    indeg[node] += 1
        q = deque([n for n, d in indeg.items() if d == 0])
        topo = []
        while q:
            u = q.popleft()
            topo.append(u)
            for v in graph[u]:
                indeg[v] -= 1
                if indeg[v] == 0:
                    q.append(v)
        if len(topo) != len(self.skills):
            return []
        return topo

In [ ]:
def total_time_for_sequence(seq: List[str], skills=SKILLS) -> int:
    return sum(skills[s]["T"] for s in seq)

# Desafio 1

In [ ]:
@timer
def challenge1_max_value_path(skills=SKILLS, target=TARGET, T_limit=350, C_limit=30, mc_runs=1000, random_seed=42):

    random.seed(random_seed)

    @memoize
    def closure(node):
        prs = skills[node]["Pre_Reqs"]
        res = set()
        for p in prs:
            res.add(p)
            res |= closure(p)
        return res

    pool = set([target]) | closure(target)

    pool_list = sorted(pool)

    @memoize
    def dp(index, chosen_fset, time_left, comp_left):
        if time_left < 0 or comp_left < 0:
            return -math.inf, frozenset()
        if index >= len(pool_list):
            # return total value
            total_v = sum(skills[s]["V"] for s in chosen_fset)
            return total_v, chosen_fset

        node = pool_list[index]
        best_v, best_set = dp(index+1, chosen_fset, time_left, comp_left)

        reqs = closure(node)
        to_add = set([node]) | reqs
        to_add_needed = to_add - set(chosen_fset)
        add_T = sum(skills[s]["T"] for s in to_add_needed)
        add_C = sum(skills[s]["C"] for s in to_add_needed)
        add_V = sum(skills[s]["V"] for s in to_add_needed)
        if add_T <= time_left and add_C <= comp_left:
            v2, set2 = dp(index+1, frozenset(set(chosen_fset) | to_add_needed), time_left - add_T, comp_left - add_C)
            if v2 > best_v:
                best_v, best_set = v2, set2

        return best_v, best_set

    det_v, det_set = dp(0, frozenset(), T_limit, C_limit)
    det_set = set(det_set)
    print("Determinístico: valor total = ", det_v, "skills:", det_set)
    mc_results = []
    @memoize
    def dp_with_values(index, chosen_fset, time_left, comp_left, values_tuple):
        if time_left < 0 or comp_left < 0:
            return -math.inf, frozenset()
        if index >= len(pool_list):
            total_v = sum(values_tuple[pool_list.index(s)] for s in chosen_fset)
            return total_v, chosen_fset

        node = pool_list[index]
        best_v, best_set = dp_with_values(index+1, chosen_fset, time_left, comp_left, values_tuple)

        reqs = closure(node)
        to_add = set([node]) | reqs
        to_add_needed = to_add - set(chosen_fset)
        add_T = sum(skills[s]["T"] for s in to_add_needed)
        add_C = sum(skills[s]["C"] for s in to_add_needed)

        if add_T <= time_left and add_C <= comp_left:
            v2, set2 = dp_with_values(index+1, frozenset(set(chosen_fset) | to_add_needed), time_left - add_T, comp_left - add_C, values_tuple)
            if v2 > best_v:
                best_v, best_set = v2, set2
        return best_v, best_set

    pool_base_values = [skills[s]["V"] for s in pool_list]
    for i in range(mc_runs):
        sampled = []
        for base in pool_base_values:
            low = base * 0.9
            high = base * 1.1
            sampled.append(random.uniform(low, high))
        vals_t = tuple(sampled)
        @memoize
        def local_dp(index, chosen_fset, time_left, comp_left):
            if time_left < 0 or comp_left < 0:
                return -math.inf, frozenset()
            if index >= len(pool_list):
                total_v = sum(vals_t[pool_list.index(s)] for s in chosen_fset)
                return total_v, chosen_fset
            node = pool_list[index]
            best_v, best_set = local_dp(index+1, chosen_fset, time_left, comp_left)
            reqs = closure(node)
            to_add = set([node]) | reqs
            to_add_needed = to_add - set(chosen_fset)
            add_T = sum(skills[s]["T"] for s in to_add_needed)
            add_C = sum(skills[s]["C"] for s in to_add_needed)
            if add_T <= time_left and add_C <= comp_left:
                v2, set2 = local_dp(index+1, frozenset(set(chosen_fset) | to_add_needed), time_left - add_T, comp_left - add_C)
                if v2 > best_v:
                    best_v, best_set = v2, set2
            return best_v, best_set

        v_mc, set_mc = local_dp(0, frozenset(), T_limit, C_limit)
        mc_results.append((v_mc, set_mc))

    vals = [r[0] for r in mc_results]
    mean_v = sum(vals)/len(vals)
    std_v = (sum((x-mean_v)**2 for x in vals)/len(vals))**0.5
    print(f"Monte Carlo ({mc_runs} runs): E[Valor] = {mean_v:.3f}, std = {std_v:.3f}")

    return {
        "deterministic": {"value": det_v, "set": det_set},
        "montecarlo": {"mean": mean_v, "std": std_v, "samples": mc_results},
        "pool_list": pool_list
    }

# Desafio 2

In [ ]:
@timer
def challenge2_permutations(skills=SKILLS, critical=CRITICAL):
    gv = GraphValidator(skills)
    orphans = gv.find_orphans()
    if orphans:
        raise ValueError(f"Found orphan prereqs: {orphans}")
    has_cycle, cycle_nodes = gv.detect_cycle()
    if has_cycle:
        raise ValueError(f"Cycle detected: {cycle_nodes}")

    perms = list(itertools.permutations(critical))
    assert len(perms) == 120

    def cost_of_order(order):
        seq = []
        learned = set()
        for skill in order:
            for p in skills[skill]["Pre_Reqs"]:
                if p not in learned:
                    for anc in GraphValidator(skills).topological_sort():
                        if anc in closure_cache(p) and anc not in learned:
                            seq.append(anc)
                            learned.add(anc)
            if skill not in learned:
                seq.append(skill)
                learned.add(skill)
        totalT = total_time_for_sequence(seq, skills)
        return totalT, seq

    @memoize
    def closure_cache_func(node):
        res = set()
        for p in skills[node]["Pre_Reqs"]:
            res.add(p)
            res |= closure_cache_func(p)
        return res
    closure_cache = closure_cache_func

    results = []
    for p in perms:
        t, seq = cost_of_order(p)
        results.append((p, t, seq))

    results_sorted = sorted(results, key=lambda x: x[1])
    top3 = results_sorted[:3]
    avg_top3 = sum(r[1] for r in top3)/3
    print("Top 3 orders (menor tempo):")
    for r in top3:
        print(r[0], "-> tempo total:", r[1], "seq final:", r[2])
    print("Custo médio entre as 3 melhores:", avg_top3)
    return {"all": results_sorted, "top3": top3, "avg_top3": avg_top3}

# Desafio 3

In [ ]:
@timer
def challenge3_greedy_vs_opt(skills=SKILLS):
    basic = [k for k, v in skills.items() if len(v["Pre_Reqs"])==0]
    target_S = 15

    greedy_order = sorted(basic, key=lambda k: skills[k]["V"]/skills[k]["T"], reverse=True)
    chosen = []
    total_v = 0
    for s in greedy_order:
        chosen.append(s)
        total_v += skills[s]["V"]
        if total_v >= target_S:
            break
    greedy_set = chosen
    greedy_time = total_time_for_sequence(greedy_set, skills)
    print("Greedy chosen:", greedy_set, "V sum:", total_v, "T total:", greedy_time)

    best = (math.inf, None, 0)
    for r in range(1, len(basic)+1):
        for comb in itertools.combinations(basic, r):
            v_sum = sum(skills[s]["V"] for s in comb)
            if v_sum >= target_S:
                t_sum = total_time_for_sequence(comb, skills)
                if t_sum < best[0]:
                    best = (t_sum, comb, v_sum)
    if best[1] is None:
        print("Não é possível atingir S>=15 apenas com básicos.")
    else:
        print("Ótimo (exaustivo): set:", best[1], "tempo:", best[0], "V:", best[2])

    toy = {
        "A": {"T": 50, "V": 60, "Pre_Reqs": []},
        "B": {"T": 10, "V": 11, "Pre_Reqs": []},
        "C": {"T": 40, "V": 40, "Pre_Reqs": []}
    }
    toy2 = {
        "A": {"T": 100, "V": 101, "Pre_Reqs": []},
        "B": {"T": 1, "V": 1, "Pre_Reqs": []},
        "C": {"T": 2, "V": 2, "Pre_Reqs": []},
        "D": {"T": 1, "V": 50, "Pre_Reqs": []}
    }
    counterexample_text = (
        "Contraexemplo (conceitual): suponha um orçamento de tempo limitado que impede escolher itens grandes.\n"
        "Greedy por V/T pode escolher um item de alta razão que ocupa quase todo o tempo e bloquear seleção de "
        "vários itens médios que em conjunto dariam maior valor total. Logo, guloso nem sempre é ótimo."
    )

    return {
        "basic": basic,
        "greedy": {"set": greedy_set, "time": greedy_time, "Vsum": total_v},
        "optimal": {"set": best[1], "time": best[0], "Vsum": best[2]},
        "counterexample": counterexample_text
    }

# Desafio 4

In [ ]:
@timer
def challenge4_sorts(skills=SKILLS):
    arr = list(skills.items())
    def merge_sort(lst):
        if len(lst) <= 1:
            return lst
        mid = len(lst)//2
        left = merge_sort(lst[:mid])
        right = merge_sort(lst[mid:])
        res = []
        i=j=0
        while i < len(left) and j < len(right):
            if left[i][1]["C"] <= right[j][1]["C"]:
                res.append(left[i]); i+=1
            else:
                res.append(right[j]); j+=1
        res.extend(left[i:])
        res.extend(right[j:])
        return res

    t0 = time.time()
    sorted_custom = merge_sort(arr)
    t1 = time.time()
    t_custom = t1 - t0

    t0 = time.time()
    sorted_native = sorted(arr, key=lambda x: x[1]["C"])
    t1 = time.time()
    t_native = t1 - t0

    ids_sorted = [k for k,_ in sorted_custom]
    sprintA = ids_sorted[:6]
    sprintB = ids_sorted[6:12]
    print("MergeSort tempo:", t_custom, "nativo tempo:", t_native)
    return {"custom_sorted": ids_sorted, "sprintA": sprintA, "sprintB": sprintB, "times": {"merge": t_custom, "native": t_native}}

# Desafio 5

In [ ]:
@timer
def challenge5_recommendation(skills=SKILLS, horizon_years=5, yearly_hours=300):
    states = ["stable","high"]
    P = {
        "stable": {"stable": 0.8, "high": 0.2},
        "high": {"stable": 0.6, "high": 0.4}
    }
    modifiers = {
        "stable": {k:1.0 for k in skills},
        "high": {k:(1.2 if k in ["S6","S4","H11"] else 1.0) for k in skills}
    }

    current = set(["S1","S2"])

    def reward(skill, state):
        base = skills[skill]["V"]
        return base * modifiers[state].get(skill,1.0)

    def available(skills, acquired):
        return [k for k,v in skills.items() if all(p in acquired for p in v["Pre_Reqs"]) and k not in acquired]

    avail = available(skills, current)
    scores = {}
    for candidate in avail:
        exp0 = 0
        for s in states:
            p0 = 0.7 if s=="stable" else 0.3
            exp0 += p0 * reward(candidate, s)
        exp1 = 0
        for s in states:
            p0 = 0.7 if s=="stable" else 0.3
            exp_next = 0
            for sp, tp in P[s].items():
                acq = set(current) | {candidate}
                av2 = [k for k in skills if k not in acq and all(p in acq for p in skills[k]["Pre_Reqs"])]
                best_r = 0
                if av2:
                    best_r = max(reward(k, sp) for k in av2)
                exp_next += tp * best_r
            exp1 += p0 * exp_next
        scores[candidate] = exp0 + 0.8*exp1
    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    top = ranked[:3]
    print("Recomendação (próximas 2-3 skills):", top)
    return {"scores": scores, "top": top}

# Rodando tudo

In [ ]:
print("=== Validação do Grafo ===")
gv = GraphValidator(SKILLS)
print("Orfãos:", gv.find_orphans())
cycle, nodes = gv.detect_cycle()
print("Ciclo encontrado?", cycle)
if cycle:
    print("Nós no ciclo:", nodes)
print("Topological Sort:", gv.topological_sort())

print("\n=== Challenge 1 ===")
res1 = challenge1_max_value_path()
print("Resultado Challenge 1:", res1)

print("\n=== Challenge 2 ===")
res2 = challenge2_permutations()
print("Top 3 permutações:", res2)

print("\n=== Challenge 3 ===")
res3 = challenge3_greedy_vs_opt()
print("Resultado Challenge 3:", res3)

print("\n=== Challenge 4 ===")
res4 = challenge4_sorts()
print("Resultado Challenge 4:", res4)

print("\n=== Challenge 5 ===")
res5 = challenge5_recommendation()
print("Resultado Challenge 5:", res5)

print("\nExecução concluída!")


=== Validação do Grafo ===
Orfãos: []
Ciclo encontrado? False
Topological Sort: ['S1', 'S2', 'S7', 'H10', 'H12', 'S3', 'S8', 'S5', 'S4', 'S9', 'S6', 'H11']

=== Challenge 1 ===
Determinístico: valor total =  18 skills: {'S1', 'S4', 'S3'}
Monte Carlo (1000 runs): E[Valor] = 17.990, std = 0.658
[TIMER] challenge1_max_value_path levou 0.0393s
Resultado Challenge 1: {'deterministic': {'value': 18, 'set': {'S1', 'S4', 'S3'}}, 'montecarlo': {'mean': 17.990056089018957, 'std': 0.6577636258020337, 'samples': [(17.058718045777056, frozenset({'S1', 'S4', 'S3'})), (19.01674931921824, frozenset({'S1', 'S4', 'S3'})), (16.844689958710227, frozenset({'S1', 'S4', 'S3'})), (17.534109393218863, frozenset({'S1', 'S4', 'S3'})), (18.452325061335216, frozenset({'S1', 'S4', 'S3'})), (18.205287530511992, frozenset({'S1', 'S4', 'S3'})), (17.393953555889983, frozenset({'S1', 'S4', 'S3'})), (18.845118300961133, frozenset({'S1', 'S4', 'S3'})), (18.489753927977308, frozenset({'S1', 'S4', 'S3'})), (18.9423014923589